In [2]:
import dash
import dash_core_components as dcc 
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
import statsmodels.api as sm
import pandas as pd 
from datetime import datetime
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/Users/siqisun/Documents/GitHub/CMM'

In [3]:
df = pd.read_csv('summary.csv')
df["week"] = df["date_val"].apply(lambda x: datetime. strptime(x,'%Y-%m-%d').strftime("%W"))
df_filt = df.dropna(subset=["pa_approved"])

provider = df['bin'].unique()
drug = df['drug'].unique()
year = df['calendar_year'].unique()
week = df["week"].unique()

tdf = df_filt[["calendar_year", "calendar_month", "week"]].drop_duplicates()
tdf['year-month'] = tdf["calendar_year"].map(str)+'-'+tdf['calendar_month'].map(str)

In [4]:
tdf

,calendar_year,calendar_month,week,year-month
0,2017,1,00,2017-1
223,2017,1,01,2017-1
7943,2017,1,02,2017-1
15490,2017,1,03,2017-1
23044,2017,1,04,2017-1
...,...,...,...,...
1296461,2019,12,48,2019-12
1305613,2019,12,49,2019-12
1314889,2019,12,50,2019-12
1324315,2019,12,51,2019-12


In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='EDA', children=[
            html.Div([
            html.Label('Drug_type'),
            html.Div([
                dcc.RadioItems(
                    id='drug_type',
                    options=[{'label': i, 'value': i} for i in drug],
                    value=drug[0],
                    labelStyle={'display': 'inline-block'})
            ]),
            html.Label('Provider type'),
            html.Div([
                dcc.RadioItems(
                    id='provider_type',
                    options=[{'label': i, 'value': i} for i in provider],
                    value=provider[0],
                    labelStyle={'display': 'inline-block'})
            ])
        ]),
        html.Div([
            dcc.Graph(id='app_rate')
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}) 
            ]),
        dcc.Tab(label='PA approval prediction', children=[
            #options, figures to be added
        ]),
        dcc.Tab(label='PA volume prediction', children=[
            html.Div([
            html.Label('Mode'),
            html.Div([
                dcc.RadioItems(
                    id='mode',
                    options=[{'label': i, 'value': i} for i in ["week", "month"]],
                    value="week",
                    labelStyle={'display': 'inline-block'})
            ]),
            html.Label('Future Prediction period'),
            html.Div([
                dcc.Input(
                    id='period',
                    type='number',
                    value=6
                ),
            ])
        ]),
        html.Div([
            dcc.Graph(id='volume_pred')
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}) 
        ]),
    ])
])


@app.callback(
    dash.dependencies.Output('app_rate', 'figure'), 
    [dash.dependencies.Input('drug_type','value'),
     dash.dependencies.Input('provider_type','value')
    ]
)

def update_app_rate(value_drug, value_provider):    
    tdf_rate = tdf.copy()
    tdf_rate = tdf_rate.set_index(["calendar_year", "calendar_month"])
    tdf['rate'] = 0
    for j in year:
        df_cal = df_filt[(df_filt['drug']==value_drug)&(df_filt['bin']==value_provider)&(df_filt['calendar_year']==j)]
        for i in range(1, 13):
            PA_requests = len(df_cal[df_cal['calendar_month']==i])
            PA_approved = len(df_cal[(df_cal['calendar_month']==i)&(df_cal['pa_approved']==1)])        
            PA_approval_rate = (PA_approved/PA_requests)*100
            tdf_rate.loc[(j, i), "rate"] = PA_approval_rate

    fig = px.line(tdf, x=tdf_rate["year-month"], y=tdf_rate["rate"])
    fig.update_layout(title='PA approval rate across time',
                   xaxis_title='Time',
                   yaxis_title='PA approval Rate')
    return fig

@app.callback(
    dash.dependencies.Output('volume_pred', 'figure'), 
    [dash.dependencies.Input('period','value'),
     dash.dependencies.Input('mode','value')
    ]
)
def update_volume_pred(value_period, value_mode):
    if value_mode == "week":
        tdf_pred = tdf.copy()
        tdf_pred = tdf_pred.set_index(["calendar_year", "week"])
        for j in year:
            df_cal = df_filt[df_filt['calendar_year']==j]
            for i in week:
                PA_requests = len(df_cal[df_cal['week']==i])
                if PA_requests != 0:
                    tdf_pred.loc[(j, i), "volume"] = PA_requests
        tdf_pred = tdf_pred.reset_index(drop=True)
        date = pd.Series([datetime. strptime(i,'%Y-%m') for i in tdf_pred["year-month"]])
        tdf_n = pd.concat([date, tdf_pred["volume"]], axis=1)
        tdf_n = tdf_n.rename(columns = {0:"year-month"})
        tdf_n.index = tdf_n["year-month"]
        #apply time shifting, best conversion!
        df_log = np.log(tdf_n["volume"])
        df_log_shift = df_log - df_log.shift()
        df_log_shift.dropna(inplace=True)
        from statsmodels.tsa.statespace.sarimax import SARIMAX
        X = df_log_shift.values
        size = int(len(X) * 0.66)
        train, test = X[0:size], X[size:len(X)]
        pred_future = list(tdf_n["year-month"][size:len(X)])
        for i in range(value_period):
            pred_future.append(pred_future[-1]+relativedelta(months=1))
        history = [x for x in train]
        predictions = []
        # walk-forward validation
        for t in range(len(pred_future)):
            model = SARIMAX(history, order=(0,1,2), seasonal_order=(0, 1, 2, 12), enforce_invertibility=False, enforce_stationarity=False)
            model_fit = model.fit(disp=-1)
            output = model_fit.forecast()
            yhat = float(output[0])
            predictions.append(yhat)
            history.append(yhat)
    else:
        tdf_pred = tdf.copy()
        tdf_pred = tdf_pred.set_index(["calendar_year", "calendar_month"])
        for j in year:
            df_cal = df_filt[df_filt['calendar_year']==j]
            for i in range(1, 13):
                PA_requests = len(df_cal[df_cal['calendar_month']==i])
                if PA_requests != 0:
                    tdf_pred.loc[(j, i), "volume"] = PA_requests
        tdf_pred = tdf_pred.reset_index(drop=True)
        date = pd.Series([datetime. strptime(i,'%Y-%m') for i in tdf_pred["year-month"]])
        tdf_n = pd.concat([date, tdf_pred["volume"]], axis=1)
        tdf_n = tdf_n.rename(columns = {0:"year-month"})
        tdf_n.index = tdf_n["year-month"]
        #apply time shifting, best conversion!
        df_log = np.log(tdf_n["volume"])
        df_log_shift = df_log - df_log.shift()
        df_log_shift.dropna(inplace=True)
        from statsmodels.tsa.arima_model import ARIMA
        X = df_log_shift.values
        size = int(len(X) * 0.66)
        train, test = X[0:size], X[size:len(X)]
        pred_future = list(tdf_n["year-month"][size:len(X)])
        for i in range(value_period):
            pred_future.append(pred_future[-1]+relativedelta(months=1))
        history = [x for x in train]
        predictions = []
        # walk-forward validation
        for t in range(len(pred_future)):
            model = ARIMA(history, order=(1,0,1))
            model_fit = model.fit(disp=-1)
            output = model_fit.forecast()
            yhat = float(output[0])
            predictions.append(yhat)
            history.append(yhat) 
        
    predictions_ARIMA_diff = pd.Series(predictions, copy=True)
    predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
    predictions_ARIMA_log = pd.Series([df_log[0:size][-1]]*len(pred_future)).add(predictions_ARIMA_diff_cumsum, fill_value=0)
    predictions_ARIMA = np.exp(predictions_ARIMA_log)
    error = sqrt(mean_squared_error(tdf_n["volume"][size:len(X)], predictions_ARIMA[:(len(X)-size)]))
    title_name = "PA volume trend, RMSE is "+ str(round(error))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_future, y=predictions_ARIMA,
                mode='lines+markers',
                name='predicted'))
    fig.add_trace(go.Scatter(x=tdf_n["year-month"], y=tdf_n["volume"],
                        mode='lines+markers',
                        name='input'))
    fig.update_layout(title=title_name,
               xaxis_title='Time',
               yaxis_title='PA volume')
    return fig



if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2021 16:08:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 16:08:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 16:08:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 16:08:27] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2021 16:08:28] "POST /_dash-update-component HTTP/1.1" 200 -
/Users/siqisun/Downloads/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

127.0.0.1 - - [20/May/2021 16:09:14] "POST /_dash-update-component HTTP/1.1" 200 -
